In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numpy as np
%matplotlib inline
import torch.optim as optim
import torchvision.utils as vutils
import torch.backends.cudnn as cudnn
print(torch.__version__)

1.7.1


In [22]:
#https://github.com/zacheberhart/Convolutional-Disentangled-Variational-Autoencoder/blob/master/Convolutional%20Disentangled%20Variational%20Autoencoder%20(%CE%B2-VAE).ipynb

In [5]:
# EPOCH = 10
# BATCH_SIZE = 32
# LR = 0.005
path = '/Users/ruchira/Documents/lvl_4_project/lvl4-hons-project/notebooks/train_test_imbal/'

In [6]:
X_train = np.loadtxt(path + 'X_train.csv', delimiter=',')
y_train = np.loadtxt(path + 'y_train.csv', delimiter=',')
X_test = np.loadtxt(path + 'X_test.csv', delimiter=',')
y_test = np.loadtxt(path + 'y_test.csv', delimiter=',')

In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(109381, 430)
(109381,)
(36461, 430)
(36461,)


In [8]:
train_arr = np.hstack((X_train, y_train.reshape(-1,1)))
print(train_arr.shape)

(109381, 431)


In [9]:
tensor_Xtrain = torch.from_numpy(X_train)
tensor_ytrain = torch.from_numpy(y_train)

train_data = Data.TensorDataset(tensor_Xtrain, tensor_ytrain)
train_loader = Data.DataLoader(dataset=train_data, batch_size=64, shuffle=True)

In [10]:
print(len(train_loader.dataset))

109381


In [23]:
CUDA = False

ZDIMS = 10
# BETA = 5
LR = 1e-3
EPOCHS = 25
BATCH_SIZE = 64
SEED = 4
LOG_INTERVAL = 100

In [24]:
torch.manual_seed(SEED)
device = torch.device('cuda' if CUDA else 'cpu')
kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}
device

device(type='cpu')

In [ ]:
NC = 1    # channels
NEF = 64  # init encoding filters
NDF = 64  # init decoding filters

class VAE(nn.Module):
    
    def __init__(self, zdims):
        super(VAE, self).__init__()
        
        self.zdims = zdims
        
        self.encoder = nn.Sequential(
            
            # input is (NC) x 28 x 28 (MNIST)
            nn.Conv1d(in_channels = NC, out_channels = NEF, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(inplace = True),

            # conv layer 2
            nn.Conv1d(in_channels = NEF, out_channels = NEF * 2, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm1d(NEF * 2),
            nn.ReLU(inplace = True),

            # conv layer 3
            nn.Conv1d(in_channels = NEF * 2, out_channels = NEF * 4, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm1d(NEF * 4),
            nn.ReLU(inplace = True),

            # conv layer 4 
            nn.Conv1d(in_channels = NEF * 4, out_channels = 1024, kernel_size = 3, stride = 1, padding = 1),
            #nn.BatchNorm2d(1024), # OPTIONAL
            nn.ReLU(inplace = True)

        )

        self.decoder = nn.Sequential(

            # input is Z (post-fc)
            nn.ConvTranspose1d(in_channels = 1024, out_channels = NDF * 8, kernel_size = 1, stride = 1),
            nn.BatchNorm1d(NDF * 8),
            nn.ReLU(inplace = True),

            # deconv layer 2
            nn.ConvTranspose1d(in_channels = NDF * 8, out_channels = NDF * 4, kernel_size = 1, stride = 1),
            nn.BatchNorm1d(NDF * 4),
            nn.ReLU(inplace = True),

            # deconv layer 3
            nn.ConvTranspose1d(in_channels = NDF * 4, out_channels = NDF * 2, kernel_size = 1, stride = 1),
            nn.BatchNorm1d(NDF * 2),
            nn.ReLU(inplace = True),

            # deconv layer 4
            nn.ConvTranspose1d(in_channels = NDF*2, out_channels = NC, kernel_size = 1, stride = 1),
            nn.Sigmoid()

        )
        
        # conv fc
        self.fc11 = nn.Linear(64, self.zdims) # mu
        self.fc12 = nn.Linear(64, self.zdims) # logvar
        
        # deconv fc
        self.fc2  = nn.Linear(self.zdims, 64)
    
    
    def encode(self, x):
#         import pdb; pdb.set_trace()
        conv = self.encoder(x)
        conv = conv.view(-1, 64) # this maybe the problem
        mu = self.fc11(conv)
        logvar = self.fc12(conv)
        return mu, logvar
    
    def decode(self, z):
#         import pdb; pdb.set_trace()
        deconv_input = F.relu(self.fc2(z))
        deconv_input = deconv_input.view(64,1024,430) # world models: [-1, 1, 1, 1024] # and this
        recon_x = self.decoder(deconv_input)
        return recon_x
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(logvar * 0.5)
        eps = torch.rand_like(std)
        z = eps.mul(std).add(mu)
        return z
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
#         print(z.shape, "z, forward")
        recon_x = self.decode(z)
        return recon_x, mu, logvar, z


In [ ]:
view_data = Variable(tensor_Xtrain[:5].view(-1, 430).type(torch.FloatTensor)/255.)


In [ ]:
model = VAE(zdims = ZDIMS).to(device)
optimizer = optim.Adam(model.parameters(), lr = LR)

In [ ]:
def loss_function(recon_x, x, mu, logvar, beta = 1):
    '''Use a beta value of 1 for a vanilla VAE'''
    print(recon_x.shape, "recon_x")
    print(x.shape, "x")
    # loss
    BCE = F.binary_cross_entropy(recon_x, x, reduction = 'sum')
    
    # KL Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return BCE + (beta * KLD)

In [ ]:
def train(epoch, beta = 1):
    
    model.train()
    train_loss = 0
    
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
#         print(data.shape)
        optimizer.zero_grad()
        recon_batch, mu, logvar, Z = model(data.view(64,1,430).type(torch.FloatTensor))
#         print(recon_batch.shape, "recon_batch")
#         print(mu.shape, "mu")
#         print(Z.shape, "Z")
        
        loss = loss_function(recon_batch, data.view(64,1,430).type(torch.FloatTensor), mu, logvar, beta)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(data),
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)
            ))
    
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch,
        train_loss / len(train_loader.dataset)
    ))

In [ ]:
for epoch in range(1, EPOCHS + 1):
    train(epoch)

In [86]:
Z = np.random.uniform(-10, 10, 70)
print(Z.shape)
X = np.random.uniform(-10, 10, 70).reshape(1, 7,-1)
print(X)
print(X.shape)

(70,)
[[[-9.02104472 -4.87034136  5.6664161   5.02689224 -3.81840654
    4.80226299  9.85344875  7.96805686 -1.46083574  8.17656189]
  [-5.73583059 -0.66716076  1.16315956 -5.63071543  5.66644351
   -7.40030951 -3.85263415  4.79828062 -5.22617693 -1.40637909]
  [-9.3395182   3.73533319 -1.14249692  7.58256298 -8.20788099
   -4.37486144 -3.43005465 -3.09681348 -8.68494841  7.9679352 ]
  [ 1.51596407  9.0995712  -6.200206   -8.26873124 -1.24161429
   -7.00308771  9.27962586  3.15391634  9.92389149 -0.76663836]
  [-8.59367473  2.89580179  1.48795639  9.89863293 -9.48271097
    3.759304    2.74061341  4.89167158  7.8500397  -8.46366966]
  [ 2.43168075 -5.35326252 -6.58658741 -9.37099351  3.42901237
    6.87150977 -5.708008    1.46227475 -4.62646994 -4.62756204]
  [ 8.23975261  9.12814875 -3.11921754  3.06148022 -4.4520757
   -1.65748303 -7.06169745  3.68722782 -5.06723375  4.22871306]]]
(1, 7, 10)
